# Exploratory Analysis Data
---

### Libraries imports

In [3]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt
import seaborn as sns

___

#### Connect to database

In [11]:
db_config = {
    "dbms_engine": os.getenv("DBMS_ENGINE"),
    "dbms_username": os.getenv("DBMS_USERNAME"),
    "dbms_password": os.getenv("DBMS_PASSWORD"),
    "dbms_ip": os.getenv("DBMS_IP"),
    "dbms_port": os.getenv("DBMS_PORT"),
    "dbms_database": os.getenv("DBMS_DATAMART_DATABASE"),
}
db_config["database_url"] = (
    f"{db_config['dbms_engine']}://{db_config['dbms_username']}:{db_config['dbms_password']}@"
    f"{db_config['dbms_ip']}:{db_config['dbms_port']}/{db_config['dbms_database']}"
)

datamart_engine = create_engine(db_config["database_url"])
try:
    with datamart_engine.connect() as conn:
        print("Connected to the database")
        # Load the data from the warehouse
        print(f"Loading data from the {os.getenv('DBMS_DATAMART_DATABASE')} database...")
        query = 'SELECT * FROM "fact_trip";'
        res = conn.execute(text(query))
        df = pd.DataFrame(res.fetchall(), columns=res.keys())
        print("Data loaded successfully")
            
except Exception as e:
    print("Error occurred during connection to the database")
    print(e)

datamart_engine.dispose()

___

#### Analyse data
##### Initial data verification

In [10]:
df.head()

   trip_id  vendor_id tpep_pickup_datetime tpep_dropoff_datetime  \
0        1          2  2024-01-02 05:32:22   2024-01-02 05:39:16   
1        2          1  2024-01-02 05:20:48   2024-01-02 05:32:18   
2        3          1  2024-01-02 05:55:43   2024-01-02 06:29:21   
3        4          2  2024-01-02 05:27:40   2024-01-02 05:31:42   
4        5          2  2024-01-02 05:55:40   2024-01-02 06:05:58   

   passenger_count  trip_distance  rate_code_id  store_and_fwd_flag  \
0              1.0           1.05           1.0               False   
1              1.0           5.70          99.0               False   
2              1.0          16.70           2.0               False   
3              1.0           1.08           1.0               False   
4              1.0           2.89           1.0               False   

   pulocationid  dolocationid  payment_type_id  fare_amount  extra  mta_tax  \
0           137           107                3          8.6    1.0      0.5   
1     

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964624 entries, 0 to 2964623
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   trip_id                int64         
 1   vendor_id              int64         
 2   tpep_pickup_datetime   datetime64[ns]
 3   tpep_dropoff_datetime  datetime64[ns]
 4   passenger_count        float64       
 5   trip_distance          float64       
 6   rate_code_id           float64       
 7   store_and_fwd_flag     bool          
 8   pulocationid           int64         
 9   dolocationid           int64         
 10  payment_type_id        int64         
 11  fare_amount            float64       
 12  extra                  float64       
 13  mta_tax                float64       
 14  tip_amount             float64       
 15  tolls_amount           float64       
 16  improvement_surcharge  float64       
 17  total_amount           float64       
 18  congestion_surcharge  

**Remarques :**
- Près de 3 millions d'entrées avec 20 colonnes 
- `passenger_count` : type `float64` n'est pas très pertinent pour un nombre de passagers => le remplacer par `int64`

In [14]:
df.describe()

,trip_id,vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,rate_code_id,pulocationid,dolocationid,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
count,2.964624e+06,2.964624e+06,2964624,2964624,2.824462e+06,2.964624e+06,2.824462e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.824462e+06,2.824462e+06
mean,1.482312e+06,1.754204e+00,2024-01-17 00:46:36.431092992,2024-01-17 01:02:13.208131584,1.339281e+00,3.652169e+00,2.069359e+00,1.660179e+02,1.651167e+02,1.161271e+00,1.817506e+01,1.451598e+00,4.833823e-01,3.335870e+00,5.270212e-01,9.756319e-01,2.680150e+01,2.256122e+00,1.411611e-01
min,1.000000e+00,1.000000e+00,2002-12-31 22:59:39,2002-12-31 23:05:41,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-8.990000e+02,-7.500000e+00,-5.000000e-01,-8.000000e+01,-8.000000e+01,-1.000000e+00,-9.000000e+02,-2.500000e+00,-1.750000e+00
25%,7.411568e+05,2.000000e+00,2024-01-09 15:59:19.750000128,2024-01-09 16:16:23,1.000000e+00,1.000000e+00,1.000000e+00,1.320000e+02,1.140000e+02,1.000000e+00,8.600000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,1.000000e+00,1.538000e+01,2.500000e+00,0.000000e+00
50%,1.482312e+06,2.000000e+00,2024-01-17 10:45:37.500000,2024-01-17 11:03:51.500000,1.000000e+00,1.680000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,1.280000e+01,1.000000e+00,5.000000e-01,2.700000e+00,0.000000e+00,1.000000e+00,2.010000e+01,2.500000e+00,0.000000e+00
75%,2.223468e+06,2.000000e+00,2024-01-24 18:23:52.249999872,2024-01-24 18:40:29,1.000000e+00,3.110000e+00,1.000000e+00,2.340000e+02,2.340000e+02,1.000000e+00,2.050000e+01,2.500000e+00,5.000000e-01,4.120000e+00,0.000000e+00,1.000000e+00,2.856000e+01,2.500000e+00,0.000000e+00
max,2.964624e+06,6.000000e+00,2024-02-01 00:01:15,2024-02-02 13:56:52,9.000000e+00,3.127223e+05,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,5.000000e+03,1.425000e+01,4.000000e+00,4.280000e+02,1.159200e+02,1.000000e+00,5.000000e+03,2.500000e+00,1.750000e+00
std,8.558134e+05,4.325902e-01,NaN,NaN,8.502817e-01,2.254626e+02,9.823219e+00,6.362391e+01,6.931535e+01,5.808686e-01,1.894955e+01,1.804102e+00,1.177600e-01,3.896551e+00,2.128310e+00,2.183645e-01,2.338558e+01,8.232747e-01,4.876239e-01


**Remarques :**
- `passenger_count` et `rate_code_id` présentent des valeurs manquantes.
- `tpep_pickup_datetime` et `tpep_dropoff_datetime` présentent des dates en 2002 alors que je suis censée avoir que des dates entre 2023 et 2024.
- La valeur max de `passenger_count` est 9, ce qui n'est pas possible pour un taxi.
- La valeur min de `trip_distance` est 0, c'est peut-être une mauvaise saisie.
- La valeur max de `rate_code_id` est 99, alors que les valeurs sont censées être comprises entre 1 et 6.
- La valeur min de `payment_type_id` est 0, alors que les valeurs sont censées être comprises entre 1 et 6.
- Les données tarifaires présentent des valeurs négatives, ce qui ne semble pas possible dans un contexte réaliste. Il faut donc investiguer sur les raisons de la présence de ces valeurs car elles sont peut-être légitimes s'il s'agit par exemple d'un remboursement.

##### Variables exploration

In [17]:
numeric_cols = list(df._get_numeric_data())
numeric_cols.remove('trip_id')

cat_cols = df.drop(numeric_cols, axis=1)

print(numeric_cols)
print(cat_cols)



['vendor_id', 'passenger_count', 'trip_distance', 'rate_code_id', 'store_and_fwd_flag', 'pulocationid', 'dolocationid', 'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee']
         trip_id tpep_pickup_datetime tpep_dropoff_datetime
0              1  2024-01-02 05:32:22   2024-01-02 05:39:16
1              2  2024-01-02 05:20:48   2024-01-02 05:32:18
2              3  2024-01-02 05:55:43   2024-01-02 06:29:21
3              4  2024-01-02 05:27:40   2024-01-02 05:31:42
4              5  2024-01-02 05:55:40   2024-01-02 06:05:58
...          ...                  ...                   ...
2964619  2964620  2024-01-02 05:43:08   2024-01-02 06:18:53
2964620  2964621  2024-01-02 05:59:39   2024-01-02 06:05:50
2964621  2964622  2024-01-02 05:28:00   2024-01-02 05:54:32
2964622  2964623  2024-01-02 05:33:53   2024-01-02 06:04:12
2964623  2964624  2024-01-02 05:32:22   2024-01-02 05:39